**Proiect Big Data - RealAgents**

---

Ion Alexandra - grupa 405




>[Introducere](#scrollTo=FaFlScXShUAA)

>>[a. Prezentarea setului de date](#scrollTo=7accJXJ4iomd)

>>[b. Obiectivele proiectului](#scrollTo=xNZTs13Bi6B0)

>[Procesarea datelor](#scrollTo=sKFm4usBhtbW)

>[Metode Machine Learning](#scrollTo=ZbK7GzM5rm-e)

>>[a. Predicția prețului de vânzare](#scrollTo=eI13arWxiKFR)

>>[b. Clasificarea caselor în "scumpe" sau "ieftine"](#scrollTo=QMIt8arqd9La)



# 1. Introducere

Proiectul a fost dezvoltat pe baza examenului final din platforma DataCamp, în vederea obținerii certificatul "Data Scientist Associate". Setul de date este stocat in fisierul "house_sales.csv".

**Tema proiectului:**

RealAgents este o companie imobiliară care vinde o varietate de tipuri de case într-o zonă metropolitană.

Unele dintre aceste case se vând lent și uneori necesită scăderea prețului pentru a găsi un cumpărător. Pentru a rămâne competitiv, RealAgents ar dori să optimizeze prețurile de listare a caselor pe care încearcă să le vândă.




## a. Prezentarea setului de date

Setul de date conține înregistrări ale caselor vândute în zonă. In tabelul de mai jos au fost detaliate informațiile privitoare la coloanele setului de date și criteriile prin care datele vor fi prelucrate.


| Numele coloanei | Criterii |
|--------------|---------------------------------------------------|
|  house_id | Nominal. </br> Identificator unic pentru case. </br>Nu sunt posibile valori lipsă. |
| city | Nominal. </br>Orașul în care se află casa. Unul dintre orasele „Silvertown”, „Riverford”, „Teasdale” și „Poppleton”. </br>Înlocuiți valorile lipsă cu „Unknown”. |
| sale_price | Discret. </br>Prețul de vânzare al casei în dolari întregi. Valorile pot fi orice număr pozitiv mai mare sau egal cu zero.</br>Eliminați intrările lipsă. |
| sale_date  | Discret. </br>Data ultimei vânzări a casei. </br>Înlocuiți valorile lipsă cu 2023-01-01. |
| months_listed  | Continuu. </br>Numărul de luni în care casa a fost listată pe piață înainte de ultima sa vânzare, rotunjit la o zecimală. </br>Înlocuiți valorile lipsă cu numărul mediu de luni afișat, cu o zecimală. |
| bedrooms | Discret. </br>Numărul de dormitoare din casă. Orice valori pozitive mai mari sau egale cu zero. </br>Înlocuiți valorile lipsă cu numărul mediu de dormitoare, rotunjit la cel mai apropiat număr întreg. |
| house_type | Ordinal. </br>Unul dintre tipurile „Terraced” (doi pereți în comun), „Semi-detached” (un perete comun) sau „Detached” (fără pereți comun). </br>Înlocuiți valorile necorespunzatoare cu cel apropiat tip de casa. |
| area | Continuu. </br>Suprafața casei în metri pătrați, rotunjită la o zecimală. </br>Înlocuiți valorile lipsă cu media, cu o zecimală. |

Mai jos se pot vedea primele 20 de înregsitrări ale setului de date,cât și schema și sumarul acestora.

In [1]:
import pandas as pd
import pyspark
from pyspark import SparkContext as sc
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import StructField, IntegerType, StringType, StructType
from pyspark.sql.functions import col, count, when, round, regexp_replace

spark = SparkSession.builder.appName("RealAgents")\
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

df = spark.read.csv('house_sales.csv',inferSchema=True,header=True)
df.show()
df.printSchema()
df.summary().show()

+--------+----------+----------+----------+-------------+--------+-------------+-----------+
|house_id|      city|sale_price| sale_date|months_listed|bedrooms|   house_type|       area|
+--------+----------+----------+----------+-------------+--------+-------------+-----------+
| 1217792|Silvertown|     55943|2021-09-12|          5.4|       2|Semi-detached|107.8 sq.m.|
| 1900913|Silvertown|    384677|2021-01-17|          6.3|       5|     Detached|498.8 sq.m.|
| 1174927| Riverford|    281707|2021-11-10|          6.9|       6|     Detached|542.5 sq.m.|
| 1773666|Silvertown|    373251|2020-04-13|          6.1|       6|         Det.|528.4 sq.m.|
| 1258487|Silvertown|    328885|2020-09-24|          8.7|       5|     Detached|477.1 sq.m.|
| 1539789| Riverford|     47143|2020-09-25|          5.1|       2|Semi-detached|123.2 sq.m.|
| 1353069|  Teasdale|    429914|2021-05-18|           NA|       6|     Detached|544.6 sq.m.|
| 1155864|  Teasdale|    284440|2020-10-15|          5.8|       4|    

## b. Obiectivele proiectului

Se doreste optimziarea prețurilor de listare ale caselor prin prezicerea în avans a acestora, având în vedere caracteristicile unei case, astfel reducând timpul pana la vanzarea ei.

**Obiectivul 1:** Predicția prețului de vânzare al caselor.

**Obiectivul 2:** Clasificarea caselor pentru a determina dacă acestea ieftine. Compania vrea să știe dacă preturile pe metru patrat ale caselor sunt prea mici.

# 2. Procesarea datelor

Pentru curățarea și transformarea datelor pe baza cerințelor din tabelul anterior se vor utiliza Dataframes si Spark SQL.

- Dupa verificarea coloanei "house_id" s-a observat că toate valorile sunt unice, așadar se respectă descrierea și nu necesită curațare.



In [2]:
df = df.dropna(subset=['house_id'])
distinct_count = df.select('house_id').distinct().count()
are_all_unique = (distinct_count == df.count())

print('Valori unice:', are_all_unique,',',df.count())


Valori unice: True , 1500


- Toate câmpurile din coloana "city" care lipseau sau nu faceau parte din lista de orașe permise au fost inlocuite cu valoarea "Unknown". In urma curățării datelor am constatat că lipseau 73 de valori.

In [4]:
#city
allowed_values = ['Silvertown', 'Riverford', 'Teasdale', 'Poppleton']

df= df.withColumn(
    'city', when(col('city').isin(allowed_values), col('city')).otherwise('Unknown'))

#df.groupby('city').count().filter((col('city') == 'Unknown')).show()
df.createOrReplaceTempView("house_sales")

spark.sql("""
SELECT city, COUNT(*) as count
FROM house_sales
WHERE city NOT IN ('Silvertown', 'Riverford', 'Teasdale', 'Poppleton')
GROUP BY city""").show()




+-------+-----+
|   city|count|
+-------+-----+
|Unknown|   73|
+-------+-----+



- După verificarea coloanei "sale_price" s-a observat ca toate valorile respectă descrierea și nu necesita curatare. Nu au fost gasite valori nule, non-integer sau negative.

In [5]:
#sale_price

null_val = df.filter(col('sale_price').isNull()).count() > 0
negative_values = df.filter(col('sale_price') < 0).count() > 0
non_integer_values = df.select('sale_price').rdd.flatMap(lambda x: x).map(lambda x: str(x).isdigit()).filter(lambda x: x == False).count() > 0

# Afișarea rezultatului
print('Valori necorespunzătoare:', null_val and negative_values and non_integer_values)


Valori necorespunzătoare: False


- Dupa verificarea coloanei "sale_date" s-a observat ca nu exista valori lipsa, asadar nu a fost inserata data '2023-01-01'.

In [6]:
#sale_date

missing_values = df.filter(col('sale_date').isNull()).count() > 0
df = df.fillna('2023-01-01', subset=['sale_date'])

print('Valori lipsă:', missing_values)

spark.sql("""
SELECT sale_date, COUNT(*) as count
FROM house_sales
WHERE sale_date = '2023-01-01'
GROUP BY sale_date
""").show()



Valori lipsă: False
+---------+-----+
|sale_date|count|
+---------+-----+
+---------+-----+



- Pentru coloana "months_listed" s-a facut media tuturor valorilor, rotunjita la o singura zecimala. Toate valorile lipsa au fost inlocuite cu media calculata anterior si au fost convertite la tipul float.

In [12]:
#months_listed

mean_months_listed = df.selectExpr('ROUND(AVG(months_listed), 1) as mean_months_listed').collect()[0]['mean_months_listed']
df = df.na.fill(mean_months_listed,subset=['months_listed'])
df = df.withColumn('months_listed', col('months_listed').cast('float'))
df.agg({'months_listed':'mean'}).show()





+------------------+
|avg(months_listed)|
+------------------+
| 5.901466671824455|
+------------------+



- Pentru coloana "bedrooms" a fost calculata media tuturor valorilor si a fost rutunjita la cel mai apropiat numar intreg. Toate valorile lipsa au fost inlocuite cu aceasta medie.

In [8]:
#bedrooms
from builtins import round
mean_bedrooms = round(df.agg({'bedrooms': 'mean'}).collect()[0][0])
print('Mean:', mean_bedrooms)
df = df.fillna(mean_bedrooms, subset=['bedrooms'])

Mean: 4


- Dupa analizarea tuturor valorilor din coloana "house_type" s-a observat ca acestea au urmatoarele tipuri: 'Det.',Detached', 'Terr.', 'Terraced', 'Semi' si 'Semi-detached'. Fiecare valoare care nu se afla in lista valorilor acceptate a fost înlocuită cu cel mai apropiat tip de casa.

In [9]:
#house_type
print("Valorile tipurilor de case inițiale:")
df.groupBy('house_type').count().show()

# Înlocuirea valorilor specifice
df = df.replace(['Det.', 'Terr.', 'Semi'], ['Detached', 'Terraced', 'Semi-detached'], 'house_type')

# Afișarea valorilor după curățare
print("Valorile tipurilor de case după curățare:")
df.groupBy('house_type').count().show()


Valorile tipurilor de case inițiale:
+-------------+-----+
|   house_type|count|
+-------------+-----+
|Semi-detached|  231|
|         Det.|   78|
|     Terraced|  123|
|     Detached| 1046|
|         Semi|   14|
|        Terr.|    8|
+-------------+-----+

Valorile tipurilor de case după curățare:
+-------------+-----+
|   house_type|count|
+-------------+-----+
|Semi-detached|  245|
|     Terraced|  131|
|     Detached| 1124|
+-------------+-----+



- Pentru coloana "area" s-a facut media tuturor valorilor, rotunjita la o singura zecimala. Toate valorile lipsa au fost inlocuite cu media calculata anterior. De asemenea, valorile numerice din aceasta coloana erau urmate de unitatea de masura 'sq.m.', la curatarea lor aceasta fiind eliminata.

In [10]:
#area
print("Inainte de tranformare:")
spark.sql("SELECT area FROM house_sales LIMIT 10").show()
df = df.withColumn('area', regexp_replace(col('area'), ' sq.m.', '').cast('float'))

mean_area = df.selectExpr('ROUND(AVG(area), 1) as mean_area').collect()[0]['mean_area']
print('Mean:', mean_area)
df = df.fillna(mean_area, subset=['area'])

df.createOrReplaceTempView("clean_data")
print("După tranformare:")
spark.sql("SELECT area FROM clean_data LIMIT 10").show()



Inainte de tranformare:
+-----------+
|       area|
+-----------+
|107.8 sq.m.|
|498.8 sq.m.|
|542.5 sq.m.|
|528.4 sq.m.|
|477.1 sq.m.|
|123.2 sq.m.|
|544.6 sq.m.|
|371.3 sq.m.|
|125.7 sq.m.|
|371.5 sq.m.|
+-----------+

Mean: 352.1
După tranformare:
+-----+
| area|
+-----+
|107.8|
|498.8|
|542.5|
|528.4|
|477.1|
|123.2|
|544.6|
|371.3|
|125.7|
|371.5|
+-----+



Datele au fost curatate si transformate si se poate observa ca nu mai existe valori nule. De asemenea, toate tipurile numerice au fost convertite la tipul int, repectiv float pentru a putea fi utilizate in cadrul metodelor de invarare automata.

In [13]:
#df.write.option("header", True).csv('clean_data.csv')
df.printSchema()
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

root
 |-- house_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- sale_price: integer (nullable = true)
 |-- sale_date: date (nullable = true)
 |-- months_listed: float (nullable = false)
 |-- bedrooms: integer (nullable = true)
 |-- house_type: string (nullable = true)
 |-- area: float (nullable = false)

+--------+----+----------+---------+-------------+--------+----------+----+
|house_id|city|sale_price|sale_date|months_listed|bedrooms|house_type|area|
+--------+----+----------+---------+-------------+--------+----------+----+
|       0|   0|         0|        0|            0|       0|         0|   0|
+--------+----+----------+---------+-------------+--------+----------+----+



# 3. Metode Machine Learning

Se va utiliza Spark MLlib pentru atingerea obiectivelor si prezicerea apretului de vanzare al unei case.

## a. Predicția prețului de vânzare

Pentru indeplinirea acestui obiectiv se va utiliza o regresie liniară, deoarece este potrivită pentru problemele de predicție bazate pe date existente.

In [18]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

if 'features' in df.columns:
    df = df.drop('features')

# s-a obtinut vectorului de caracteristici pe baza coloanelor 'months_listed', 'bedrooms' si 'area'
assembler = VectorAssembler(inputCols=['months_listed', 'bedrooms', 'area'], outputCol='features')
output = assembler.transform(df)

#afisarea coloanei obtinute
output.select("features").show(truncate=False)


+------------------------------------------+
|features                                  |
+------------------------------------------+
|[5.400000095367432,2.0,107.80000305175781]|
|[6.300000190734863,5.0,498.79998779296875]|
|[6.900000095367432,6.0,542.5]             |
|[6.099999904632568,6.0,528.4000244140625] |
|[8.699999809265137,5.0,477.1000061035156] |
|[5.099999904632568,2.0,123.19999694824219]|
|[5.900000095367432,6.0,544.5999755859375] |
|[5.800000190734863,4.0,371.29998779296875]|
|[4.400000095367432,2.0,125.69999694824219]|
|[3.0,4.0,371.5]                           |
|[4.599999904632568,5.0,428.79998779296875]|
|[3.700000047683716,2.0,147.6999969482422] |
|[5.800000190734863,4.0,395.6000061035156] |
|[6.199999809265137,4.0,304.3999938964844] |
|[5.300000190734863,4.0,316.79998779296875]|
|[3.5,6.0,554.2999877929688]               |
|[5.0,4.0,374.8999938964844]               |
|[4.599999904632568,3.0,294.29998779296875]|
|[5.5,4.0,397.0]                           |
|[6.900000

In [19]:
# s-a creat o diviziune train-test cu procentele 70%-30%
train_data, test_data =output.randomSplit([0.7, 0.3])
train_data.summary().show()
test_data.summary().show()



+-------+------------------+---------+------------------+------------------+------------------+----------+------------------+
|summary|          house_id|     city|        sale_price|     months_listed|          bedrooms|house_type|              area|
+-------+------------------+---------+------------------+------------------+------------------+----------+------------------+
|  count|              1035|     1035|              1035|              1035|              1035|      1035|              1035|
|   mean|1498899.9777777777|     NULL| 227546.2193236715| 5.854879234551231|4.0028985507246375|      NULL|350.58560352417584|
| stddev| 287454.2389989814|     NULL|118080.83710000177|1.9295730881851567|1.4124999116234962|      NULL|146.37819390077433|
|    min|           1000296|Poppleton|             26348|               0.6|                 2|  Detached|             100.3|
|    25%|           1245874|     NULL|            128097|               4.5|                 3|      NULL|            

In [20]:
# crearea si antrenarea modelului de regresie liniară
lr = LinearRegression(labelCol='sale_price', featuresCol='features',predictionCol='prediction')
lr_model = lr.fit(train_data)
predictions = lr_model.transform(test_data)

# coeficienții și intercepția pentru regresia liniară

print("Coefficients: {}".format(str(lr_model.coefficients)))
print('\n')
print("Intercept:{}".format(str(lr_model.intercept)))

#evaluarea modelul pe datele de test
test_results=lr_model.evaluate(test_data)
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

# Sumarizarea modelului pentru setul de training și afișarea unor metrici
trainingSummary = lr_model.summary
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))





Coefficients: [-457.75243950221795,-4362.826532683838,795.4135339408286]


Intercept:-31170.277266921592
+-------------------+
|          residuals|
+-------------------+
|-26989.462367805478|
|  50642.34339688916|
|  28357.08715322576|
|  -86665.5188716785|
|  25508.07220620726|
|-29751.909303251654|
|-30776.009159311594|
| 15802.772014502407|
| -46382.92675871967|
|  21884.53478070663|
|-3286.1023053935496|
|  37908.96885499076|
|  8111.687222196357|
|-2558.4137548044673|
|  36131.16042684775|
|-30159.096147269825|
|  16822.15270712183|
|  7144.544802900913|
| -50439.99596027442|
|  40564.83318800866|
+-------------------+
only showing top 20 rows

RMSE: 40366.68695257054
+-------------------+
|          residuals|
+-------------------+
|  6102.219937545073|
|-46962.468089436006|
| 13412.272032380977|
|  48611.99551800551|
|  48233.47621036548|
| -34898.47842202976|
| -67443.72883518884|
| 19541.898972549185|
|-11851.719491028707|
|  28237.57652307584|
| -2975.136561735533|
| -77094.

In [21]:
# s-a aplicat modelul obtinut anterior pe datele de test neetichetate
unlabled_data=test_data.select('features')
predictions=lr_model.transform(unlabled_data)

#predicțiile obținute
predictions.show(),test_data.select("features","sale_price").show()
#Afișați alte informații disponibile în test_results
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print(f"R2: {test_results.r2}")

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.90000009536743...| 306148.4623678055|
|[7.0,6.0,537.2000...|366744.65660311084|
|[6.19999980926513...|219855.91284677424|
|[6.09999990463256...| 383303.5188716785|
|[7.19999980926513...|190126.92779379274|
|[4.69999980926513...|390784.90930325165|
|[8.19999980926513...| 119979.0091593116|
|[8.19999980926513...| 186885.2279854976|
|[6.40000009536743...|392233.92675871967|
|[8.80000019073486...|177610.46521929337|
|[5.80000019073486...|405076.10230539355|
|[4.90000009536743...|219576.03114500924|
|[5.90000009536743...| 45853.31277780364|
|[9.39999961853027...| 48546.41375480447|
|[2.29999995231628...|318295.83957315225|
|[7.69999980926513...|160058.09614726983|
|[6.19999980926513...|121530.84729287817|
|[5.0,2.0,140.6000...| 69650.45519709909|
|[3.40000009536743...|227659.99596027442|
|[9.10000038146972...|182325.16681199134|
+--------------------+------------

## b. Clasificarea prețului caselor

 Pentru indeplinirea acestui obiectiv se va utiliza metoda regresiei logistice, deoarece este potrivită pentru probleme de clasificare binară. Pentru rezolvarea acestei metode se va utiliza un Pipeline.

 Dorim sa clasificăm casele astfel încât să stabilim dacă o casă este ieftina. Astfel vom definii o casă ca fiind ieftină dacă prețul pe metru pătrat al acesteia este sub prețul mediu pe metru pătrat al tuturor caselor.

 Se va calcula media pe metru patrat si se va adauga o noua coloana cu valorile 1 sau 0, unde valoarea 1 inseamna ca o casa este ieftina.

In [21]:
from pyspark.sql.functions import mean, when

# copierea datelor initiale
original_df = df.select("*")

#calcularea mediei
average_price_per_sqft = df.selectExpr('ROUND(AVG(sale_price/area), 1) as average_price_per_sqft').collect()[0]['average_price_per_sqft']
print("Media pretului/metru:",average_price_per_sqft)

# Adăugarea coloanei cu eticheta
df = df.withColumn('cheap', when(col('sale_price')/col('area') < average_price_per_sqft, 1).otherwise(0))
df.show()

Media pretului/metru: 617.0
+--------+----------+----------+----------+-------------+--------+-------------+-----+-----+
|house_id|      city|sale_price| sale_date|months_listed|bedrooms|   house_type| area|cheap|
+--------+----------+----------+----------+-------------+--------+-------------+-----+-----+
| 1217792|Silvertown|     55943|2021-09-12|          5.4|       2|Semi-detached|107.8|    1|
| 1900913|Silvertown|    384677|2021-01-17|          6.3|       5|     Detached|498.8|    0|
| 1174927| Riverford|    281707|2021-11-10|          6.9|       6|     Detached|542.5|    1|
| 1773666|Silvertown|    373251|2020-04-13|          6.1|       6|     Detached|528.4|    0|
| 1258487|Silvertown|    328885|2020-09-24|          8.7|       5|     Detached|477.1|    0|
| 1539789| Riverford|     47143|2020-09-25|          5.1|       2|Semi-detached|123.2|    1|
| 1353069|  Teasdale|    429914|2021-05-18|          5.9|       6|     Detached|544.6|    0|
| 1155864|  Teasdale|    284440|2020-10-15

In [16]:
from pyspark.ml.feature import (VectorAssembler, OneHotEncoder, StringIndexer)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

if 'encoded_features' in df.columns:
    df = df.drop('encoded_features')

#afisarea coloanelor
print(df.columns)


# transformaea coloanelor categoricale
house_type_indexer = StringIndexer(inputCol='house_type', outputCol='house_type_index')
house_type_encoder = OneHotEncoder(inputCol='house_type_index', outputCol='house_type_vec')


# Asamblarea coloanelor și crearea modelului
assembler = VectorAssembler(inputCols=['house_type_vec','sale_price','area'], outputCol='encoded_features')
log_reg = LogisticRegression(featuresCol='encoded_features', labelCol='cheap')

# Inlantuirea index-urilor și encodere-lor create anterior împreună cu variabila assembler și regresia, ca etape în cadrul unui Pipeline
pipeline = Pipeline(stages=[house_type_indexer, house_type_encoder,
                            assembler, log_reg])

# Obținerea datelor de train și test printr-o diviziune cu procentele 70%-30%
train_lr_data, test_lr_data = df.randomSplit([0.7, 0.3])

# Antrenarea pipeline-ului
fit_lr_model = pipeline.fit(train_lr_data)

# Aplicare model pe datele de test
lr_results = fit_lr_model.transform(test_lr_data)

lr_results.show()


['house_id', 'city', 'sale_price', 'sale_date', 'months_listed', 'bedrooms', 'house_type', 'area', 'cheap']
+--------+----------+----------+----------+-------------+--------+-------------+-----+-----+----------------+--------------+--------------------+--------------------+--------------------+----------+
|house_id|      city|sale_price| sale_date|months_listed|bedrooms|   house_type| area|cheap|house_type_index|house_type_vec|    encoded_features|       rawPrediction|         probability|prediction|
+--------+----------+----------+----------+-------------+--------+-------------+-----+-----+----------------+--------------+--------------------+--------------------+--------------------+----------+
| 1001618| Poppleton|    282662|2020-08-15|          2.4|       5|     Detached|482.4|    1|             0.0| (2,[0],[1.0])|[1.0,0.0,282662.0...|[-4112.7657214778...|           [0.0,1.0]|       1.0|
| 1005837| Poppleton|    279159|2021-05-17|          5.9|       5|     Detached|454.9|    1|    

In [20]:
# Evaluarea modelului
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='cheap')
lr_results.select('cheap','prediction').show()

AUC = eval.evaluate(lr_results)

print('AUC:',AUC)

+-----+----------+
|cheap|prediction|
+-----+----------+
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
+-----+----------+
only showing top 20 rows

AUC: 0.9980916030534351


In [24]:
# s-a aplicat modelul obtinut anterior pe datele initiale, neetichetate
lr_predictions=fit_lr_model.transform(original_df)
lr_predictions.select('cheap','prediction').show()

+-----+----------+
|cheap|prediction|
+-----+----------+
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
+-----+----------+
only showing top 20 rows

